In [2]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='AC_PROGRESS'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '0 1 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [2]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")

In [1]:
# from_tb = "OM_Delivery"

In [3]:
table_name = "d_accumulated_progress"
table_temp = "d_accumulated_progress_temp"

In [46]:
dsql = \
f"""
TRUNCATE TABLE biteam.{table_name}
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)

delete_sql:  
TRUNCATE TABLE biteam.d_accumulated_progress



In [44]:
cn_lst = ['MR0001','MR0005','MR0010','MR0011','MR0012','MR0013','MR0014','MR0015','MR0016'] 

for cn in cn_lst:
	sql = \
	f"""
	DECLARE @LangID SMALLINT=1;
	DECLARE @FromDate DATETIME ='2022-01-01';
	DECLARE @ToDate DATETIME ='2022-05-24';

	DECLARE @BranchID  VARCHAR(30)='{cn}';
	DECLARE @AccumulateID VARCHAR(50)='CSBH2204OTC-14QD/MR-KS-STO';

	DECLARE @AccumulateSelected1 nvarchar(400)= dbo.fr_Language('AccumulateSelected1', @langid);
	DECLARE @Selected BIT = 1;

	DECLARE @AccumulateSelected0 nvarchar(400)= dbo.fr_Language('AccumulateSelected0', @langid);
	DECLARE @ApplyType VARCHAR(5)=(select TOP 1 ApplyType from OM_Accumulated a WITH(NOLOCK) WHERE a.AccumulateID=@AccumulateID);


	with #ReturnOrd as
	(
	SELECT DISTINCT o.BranchID, o.OrderNbr ,OrigOrderNbrCM
	--INTO #ReturnOrd
	FROM (
	-- Đơn Trả CO WHERE THEO pdasaleord od.OriOrderNbrUp
	SELECT DISTINCT o.BranchID, o.OrigOrderNbr AS OrderNbr,oc.OrigOrderNbr AS OrigOrderNbrCM
	--INTO #ReturnOrd
	FROM dbo.OM_SalesOrd o WITH(NOLOCK)
	INNER JOIN dbo.OM_PDASalesOrd od WITH(NOLOCK) ON od.BranchID = o.BranchID AND o.OrigOrderNbr = od.OrderNbr
	INNER JOIN dbo.OM_SalesOrd oc WITH(NOLOCK) ON oc.BranchID = od.BranchID AND oc.OrderNbr = od.OriOrderNbrUp
			
	WHERE o.Status ='C' AND o.OrderType IN ('CO') AND o.BranchID=@BranchID and (od.OrderDate BETWEEN @FromDate AND @ToDate)
	--AND o.OrderNbr='HD092021-09682'	
	UNION
	-- Đơn Trả IR WHERE THEO pdasaleord od.OrigOrderNbr
	SELECT DISTINCT o.BranchID, o.OrigOrderNbr AS OrderNbr,oc.OrigOrderNbr AS OrigOrderNbrCM
	--INTO #ReturnOrd
	FROM dbo.OM_SalesOrd o WITH(NOLOCK)
	INNER JOIN dbo.OM_PDASalesOrd od WITH(NOLOCK) ON od.BranchID = o.BranchID AND o.OrigOrderNbr = od.OrderNbr
	INNER JOIN dbo.OM_SalesOrd oc WITH(NOLOCK) ON oc.BranchID = od.BranchID AND oc.OrderNbr = od.OrigOrderNbr
	--INNER JOIN #tblAccumulatedResult b  WITH(NOLOCK) ON b.CustID = o.CustID AND (od.OrderDate BETWEEN b.FromDate AND b.ToDate)
	WHERE o.Status ='C'AND o.OrderType IN ('IR') AND o.BranchID=@BranchID and (od.OrderDate BETWEEN @FromDate AND @ToDate)
	) o	
	)
	-- Đơn Bán

	,
	#SalesOrdC as
	(
	SELECT DISTINCT o.BranchID, o.OrigOrderNbr AS OrderNbr
	--INTO #SalesOrdC
	FROM OM_SalesOrd o WITH(NOLOCK)
	INNER JOIN dbo.OM_PDASalesOrd od WITH(NOLOCK) ON od.BranchID = o.BranchID AND o.OrigOrderNbr = od.OrderNbr
	--INNER JOIN #tblAccumulatedResult b  WITH(NOLOCK) ON b.CustID = o.CustID AND (od.OrderDate BETWEEN b.FromDate AND b.ToDate)
	LEFT JOIN OM_AccumulatedResultDet ad WITH(NOLOCK) ON od.BranchID =ad.BranchID AND od.OrderNbr=ad.OrderNbr
	WHERE o.OrderType ='IN' 
	AND ad.BranchID IS NULL --20211030 trunght loại các đơn đã có trong OM_AccumulatedResultDet
	AND o.BranchID = @BranchID AND o.Status = 'C'
	and (od.OrderDate BETWEEN @FromDate AND @ToDate)

	)
	---20210927 TRUNGHT VIẾT LẠI LẤY ĐƠN DET
	-- Đơn Trả
	SELECT T.ARDocType,T.AccumulateSelected,T.LineRef,T.AccumulateID,T.BranchID,T.CustID,T.OrderNbr,
	T.AccumulatedValue,T.OrderDate,T.tstamp,T.Sel,T.OrderType,SumDiscAmt = SUM(ROUND(t.SumDiscAmt,0)) FROM 
	(
	SELECT
		ot.ARDocType,--20210629 trunght lấy thêm ARDocType để kiểm tra ko kiểm tra theo OrderType
		AccumulateSelected = CASE WHEN d.Selected = @Selected 
								THEN @AccumulateSelected1
								ELSE @AccumulateSelected0 
							END, 
		LineRef ='', 
		d.AccumulateID,
		a.BranchID,a.CustID,a.OrderNbr
		,AccumulatedValue =  SUM (
						CASE WHEN @ApplyType = 'A' THEN c.LineQty * c.SlsPrice * -1 
						ELSE (
								CASE WHEN c.UnitMultDiv = 'M' THEN c.LineQty * UnitRate 
								ELSE c.LineQty / c.UnitRate 
								END
						) END)--a.LineAmt--20210927 TRUNGHT SUM lại tránh bị double 2 dòng accumulate giống nhau
		,a.OrderDate,
		SumDiscAmt = ROUND(od.DiscAmt,0) * -1,  --- Ngochb bỏ sum do trùng kết quả
		d.tstamp,
		Sel=d.Selected,
		a.OrderType
	FROM OM_PDASalesOrd a WITH(NOLOCK)
	INNER JOIN OM_PDASalesOrdDet c WITH(NOLOCK) ON c.BranchID = a.BranchID AND c.OrderNbr = a.OrderNbr AND c.FreeItem = 0
	INNER JOIN OM_AccumulatedInvtSetup T11 ON T11.AccumulateID = @AccumulateID AND T11.InvtID = c.InvtID
	INNER JOIN #ReturnOrd oc ON oc.OrderNbr = a.OrderNbr AND oc.BranchID = a.BranchID
	INNER JOIN OM_OrderType ot WITH(NOLOCK) ON a.OrderType = ot.OrderType		
	INNER JOIN OM_AccumulatedOrderApproval d WITH(NOLOCK) ON d.OrderNbr = a.OrderNbr AND d.BranchID = a.BranchID AND d.AccumulateID = @AccumulateID	
	LEFT JOIN OM_PDAOrdDisc od WITH(NOLOCK) ON od.OrderNbr = a.OrderNbr AND od.BranchID = a.BranchID AND c.LineRef IN (SELECT part FROM dbo.fr_SplitStringMAX(od.GroupRefLineRef,','))
	WHERE a.BranchID = @BranchID AND a.Status = 'C' AND ot.ARDocType ='CM'
	GROUP BY od.DiscAmt,ot.ARDocType,d.AccumulateID,a.BranchID,a.CustID,a.OrderNbr,a.LineAmt,a.OrderDate, d.tstamp, d.Selected, a.OrderType --- Ngochb bỏ sum do trùng kết quả
	)T
	--LEFT JOIN OM_PDAOrdDisc od WITH(NOLOCK) ON od.OrderNbr = T.OrderNbr AND od.BranchID = T.BranchID --AND T.LineRef IN (SELECT part FROM dbo.fr_SplitStringMAX(od.GroupRefLineRef,','))
	GROUP BY T.ARDocType,T.AccumulateSelected,T.LineRef,T.AccumulateID,T.BranchID,T.CustID,T.OrderNbr,T.AccumulatedValue,T.OrderDate,T.tstamp,T.Sel,T.OrderType
	UNION ALL
	--ĐƠN BÁN
	SELECT T.ARDocType,T.AccumulateSelected,T.LineRef,T.AccumulateID,T.BranchID,T.CustID,T.OrderNbr,
	T.AccumulatedValue,T.OrderDate,T.tstamp,T.Sel,T.OrderType,SumDiscAmt = SUM(ROUND(t.SumDiscAmt,0))
	FROM
	(
	SELECT
			ot.ARDocType,--20210629 trunght lấy thêm ARDocType để kiểm tra ko kiểm tra theo OrderType
		AccumulateSelected = CASE WHEN d.Selected = @Selected 
							THEN @AccumulateSelected1
							ELSE @AccumulateSelected0 
						END, 
		LineRef ='', 
		d.AccumulateID,
		a.BranchID,a.CustID,a.OrderNbr
		,AccumulatedValue = SUM (
						CASE WHEN @ApplyType = 'A' THEN c.LineQty * c.SlsPrice 
						ELSE (
								CASE WHEN c.UnitMultDiv = 'M' THEN c.LineQty * UnitRate 
								ELSE c.LineQty / c.UnitRate 
								END
						) END)--a.LineAmt--20210927 TRUNGHT SUM lại tránh bị double 2 dòng accumulate giống nhau
		,a.OrderDate,
		SumDiscAmt = ROUND(ISNULL(od.DiscAmt,0),0),--SUM(ROUND(ISNULL(od.DiscAmt,0),0)),  --- Ngochb bỏ sum do trùng kết quả
		d.tstamp,
		Sel=d.Selected,
		a.OrderType

	FROM OM_PDASalesOrd a WITH(NOLOCK)
	INNER JOIN OM_PDASalesOrdDet c WITH(NOLOCK) ON c.BranchID = a.BranchID AND c.OrderNbr = a.OrderNbr AND c.FreeItem = 0
	INNER JOIN OM_AccumulatedInvtSetup T11 ON T11.AccumulateID = @AccumulateID AND T11.InvtID = c.InvtID
	INNER JOIN #SalesOrdC oc ON oc.OrderNbr = a.OrderNbr AND oc.BranchID = a.BranchID
	INNER JOIN OM_OrderType ot WITH(NOLOCK) ON a.OrderType = ot.OrderType		
	INNER JOIN OM_AccumulatedOrderApproval d WITH(NOLOCK) ON d.OrderNbr = a.OrderNbr AND d.BranchID = a.BranchID AND d.AccumulateID = @AccumulateID	
	LEFT JOIN OM_PDAOrdDisc od WITH(NOLOCK) ON od.OrderNbr = a.OrderNbr AND od.BranchID = a.BranchID AND c.LineRef IN (SELECT part FROM dbo.fr_SplitStringMAX(od.GroupRefLineRef,','))
	WHERE a.BranchID = @BranchID AND a.Status = 'C' AND ot.ARDocType ='IN'
	GROUP BY ISNULL(od.DiscAmt,0),ot.ARDocType,d.AccumulateID,a.BranchID,a.CustID,a.OrderNbr,a.LineAmt,a.OrderDate, d.tstamp, d.Selected, a.OrderType--,od.DiscAmt --- Ngochb bỏ sum do trùng kết quả
	--ORDER BY d.Selected DESC
	)T
	--LEFT JOIN OM_PDAOrdDisc od WITH(NOLOCK) ON od.OrderNbr = T.OrderNbr AND od.BranchID = T.BranchID --AND T.LineRef IN (SELECT part FROM dbo.fr_SplitStringMAX(od.GroupRefLineRef,','))
	GROUP BY T.ARDocType,T.AccumulateSelected,T.LineRef,T.AccumulateID,T.BranchID,T.CustID,T.OrderNbr,T.AccumulatedValue,T.OrderDate,T.tstamp,T.Sel,T.OrderType
	--DROP TABLE #tblAccumulatedResult
	--DROP TABLE #tblOrderDet
	--DROP TABLE #SalesOrdC
	--DROP TABLE #ReturnOrd
	"""
	df = get_ms_df(sql)
	try:
		assert df.shape[0] > 0
		df.columns = lower_col(df)
		drop_cols(df, 'tstamp')
		df['inserted_at'] = datetime.now()
		df.columns = ['ardoctype', 'accumulateselected', 'lineref', 'accumulateid',
       'branchid', 'custid', 'origordernbr', 'accumulatedvalue', 'orderdate',
       'sel', 'ordertype', 'sumdiscamt','inserted_at']
		bq_values_insert(df, f"{table_name}", 2)
		print("finish insert at cn", cn)
	except AssertionError:
		print("No DATA AT CN", cn)


1it [00:04,  4.30s/it]


No DATA AT CN MR0005


1it [00:03,  3.19s/it]


No DATA AT CN MR0011


1it [00:06,  6.42s/it]
1it [00:03,  3.40s/it]
1it [00:05,  5.30s/it]
1it [00:03,  3.08s/it]
1it [00:05,  5.36s/it]


In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update